### Imports

In [1]:
from pprint import pprint

import numpy as np

In [2]:
from rayuela.base.semiring import Boolean, Real, Tropical
from rayuela.base.symbol import Sym, ε
from rayuela.fsa.fsa import FSA
from rayuela.fsa.state import State
from rayuela.fsa.pathsum import Pathsum, Strategy

## Example 1

In [8]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(3), Sym('a'), State(6))
fsa.add_arc(State(3), Sym('b'), State(1))

fsa.add_arc(State(6), Sym('c'), State(5))

fsa.add_arc(State(1), Sym('c'), State(5))
fsa.add_arc(State(1), Sym('b'), State(4))

fsa.add_arc(State(4), Sym('a'), State(2))

fsa.add_arc(State(5), Sym('a'), State(2))

fsa.set_I(State(3))
fsa.set_F(State(2))

fsa

WFSA(6 states, <class 'rayuela.base.semiring.Boolean'>)

In [13]:
list(fsa.toposort())

[3, 1, 4, 6, 5, 2]

## Example 2

Backward with all the intermediate β values

In [90]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(1))

fsa.add_arc(State(3), Sym('a'), State(6), Real(1))
fsa.add_arc(State(3), Sym('b'), State(1), Real(7))

fsa.add_arc(State(6), Sym('c'), State(5), Real(4))

fsa.add_arc(State(1), Sym('c'), State(5), Real(3))
fsa.add_arc(State(1), Sym('b'), State(4), Real(1))

fsa.add_arc(State(4), Sym('a'), State(2), Real(5))

fsa.add_arc(State(5), Sym('a'), State(2), Real(3))
fsa.add_arc(State(5), Sym('a'), State(7), Real(8))

fsa.add_arc(State(6), Sym('a'), State(7), Real(7))

fsa.add_arc(State(7), Sym('a'), State(8), Real(2))

fsa.set_I(State(3), Real(3))
fsa.set_F(State(2), Real(4))
fsa.set_F(State(7), Real(2))

fsa

WFSA(8 states, <class 'rayuela.base.semiring.Real'>)

In [91]:
pprint(Pathsum(fsa).backward(strategy=Strategy.VITERBI))

{8: 0.0,
 4: 20.0,
 5: 28.0,
 1: 104.0,
 6: 126.0,
 7: 2.0,
 2: 4.0,
 3: 854.0}


In [92]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.VITERBI))

2562.0


## Example 3

Matrix closure by fixed point

We can compute $\left( I - M\right)^{-1}$ using the fixed point algorithm ran until numerical convergence.

In [37]:
D = 5

M = np.random.rand(D, D) / D  # ensure entries are smaller than 1 / D
M

array([[0.01027232, 0.06327649, 0.04756849, 0.02031754, 0.12113094],
       [0.0096312 , 0.03479378, 0.15765437, 0.16380315, 0.03878034],
       [0.19954012, 0.15323823, 0.09465078, 0.01604716, 0.11936823],
       [0.03476662, 0.07260129, 0.15169991, 0.05674668, 0.13927935],
       [0.03732759, 0.00609119, 0.09093126, 0.05166527, 0.01201713]])

In [40]:
R = np.zeros_like(M)
for i in range(200):
    R = (np.eye(D, D) + R) @ M
np.eye(D, D) + R

array([[1.03712624, 0.08715881, 0.09247988, 0.04717713, 0.14840145],
       [0.07001464, 1.0944272 , 0.23896784, 0.20159041, 0.10883362],
       [0.25101744, 0.21146274, 1.1861166 , 0.07286024, 0.1926543 ],
       [0.09397236, 0.12679368, 0.23119859, 1.09905688, 0.19936961],
       [0.06763309, 0.03613342, 0.12622452, 0.06720489, 1.04659829]])

In [39]:
np.linalg.inv(np.eye(D, D) - M)

array([[1.03712624, 0.08715881, 0.09247988, 0.04717713, 0.14840145],
       [0.07001464, 1.0944272 , 0.23896784, 0.20159041, 0.10883362],
       [0.25101744, 0.21146274, 1.1861166 , 0.07286024, 0.1926543 ],
       [0.09397236, 0.12679368, 0.23119859, 1.09905688, 0.19936961],
       [0.06763309, 0.03613342, 0.12622452, 0.06720489, 1.04659829]])

Matrix power in logarithmic number of multiplications

In [56]:
M = np.random.rand(D, D)

In [63]:
m = 13
# 13 = 8 + 4 + 1 = 2^3 + 2^2 + 2^0

In [85]:
Ms = [M]
for r in range(1, 4):
    Ms.append(Ms[-1] @ Ms[-1])

In [86]:
Ms[0] @ Ms[2] @ Ms[3]

array([[6.13840964e-05, 5.41136168e-05, 6.08701107e-05, 5.83428147e-05,
        6.36730990e-05],
       [6.03812196e-05, 5.32295208e-05, 5.98756301e-05, 5.73896245e-05,
        6.26328226e-05],
       [5.23322003e-05, 4.61338484e-05, 5.18940081e-05, 4.97393952e-05,
        5.42836586e-05],
       [6.93359178e-05, 6.11236031e-05, 6.87553447e-05, 6.59006566e-05,
        7.19214397e-05],
       [6.54939891e-05, 5.77367201e-05, 6.49455869e-05, 6.22490781e-05,
        6.79362452e-05]])

In [87]:
np.linalg.matrix_power(M, 13)

array([[6.13840964e-05, 5.41136168e-05, 6.08701107e-05, 5.83428147e-05,
        6.36730990e-05],
       [6.03812196e-05, 5.32295208e-05, 5.98756301e-05, 5.73896245e-05,
        6.26328226e-05],
       [5.23322003e-05, 4.61338484e-05, 5.18940081e-05, 4.97393952e-05,
        5.42836586e-05],
       [6.93359178e-05, 6.11236031e-05, 6.87553447e-05, 6.59006566e-05,
        7.19214397e-05],
       [6.54939891e-05, 5.77367201e-05, 6.49455869e-05, 6.22490781e-05,
        6.79362452e-05]])

### Graph closure by WFK

The closure of the graph answers the question whether there is a path between pairs of nodes for each possible pair

#### Acyclic

In [51]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(2), Sym('a'), State(8))

fsa.add_arc(State(3), Sym('c'), State(5))
fsa.add_arc(State(3), Sym('a'), State(7))

fsa.add_arc(State(5), Sym('a'), State(2))
fsa.add_arc(State(5), Sym('a'), State(7))

fsa.set_I(State(3))
fsa.set_F(State(2))
fsa.set_F(State(7))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Boolean'>)

In [52]:
pprint(Pathsum(fsa).lehmann())

{(3, 8): True,
 (2, 2): True,
 (8, 7): False,
 (7, 8): False,
 (7, 7): True,
 (7, 3): False,
 (3, 7): True,
 (3, 3): True,
 (2, 7): False,
 (2, 3): False,
 (2, 5): False,
 (7, 2): False,
 (5, 3): False,
 (8, 5): False,
 (5, 8): True,
 (8, 3): False,
 (7, 5): False,
 (3, 5): True,
 (5, 7): True,
 (8, 2): False,
 (8, 8): True,
 (2, 8): True,
 (5, 2): True,
 (3, 2): True,
 (5, 5): True}


#### A self-loop

In [53]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(2), Sym('a'), State(8))

fsa.add_arc(State(3), Sym('c'), State(5))
fsa.add_arc(State(3), Sym('a'), State(7))

fsa.add_arc(State(5), Sym('a'), State(2))
fsa.add_arc(State(5), Sym('a'), State(5))
fsa.add_arc(State(5), Sym('a'), State(7))

fsa.set_I(State(3))
fsa.set_F(State(2))
fsa.set_F(State(7))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Boolean'>)

In [54]:
pprint(Pathsum(fsa).lehmann())

{(8, 7): False,
 (3, 2): True,
 (3, 8): True,
 (2, 2): True,
 (7, 8): False,
 (5, 5): True,
 (8, 8): True,
 (2, 3): False,
 (8, 5): False,
 (3, 7): True,
 (7, 5): False,
 (2, 7): False,
 (7, 2): False,
 (5, 8): True,
 (3, 5): True,
 (7, 7): True,
 (2, 5): False,
 (2, 8): True,
 (5, 3): False,
 (5, 7): True,
 (3, 3): True,
 (5, 2): True,
 (8, 2): False,
 (8, 3): False,
 (7, 3): False}


#### A cycle

In [56]:
fsa = FSA(R=Boolean)

fsa.add_arc(State(2), Sym('a'), State(8))

fsa.add_arc(State(3), Sym('c'), State(5))
fsa.add_arc(State(3), Sym('a'), State(7))

fsa.add_arc(State(5), Sym('a'), State(2))

fsa.add_arc(State(7), Sym('a'), State(5))

fsa.add_arc(State(8), Sym('a'), State(7))

fsa.set_I(State(3))
fsa.set_F(State(2))
fsa.set_F(State(7))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Boolean'>)

In [57]:
pprint(Pathsum(fsa).lehmann())

{(2, 2): True,
 (3, 8): True,
 (7, 8): True,
 (3, 2): True,
 (7, 5): True,
 (5, 2): True,
 (8, 8): True,
 (5, 7): True,
 (5, 8): True,
 (5, 5): True,
 (8, 2): True,
 (7, 2): True,
 (2, 7): True,
 (2, 5): True,
 (3, 5): True,
 (3, 3): True,
 (2, 8): True,
 (3, 7): True,
 (8, 3): False,
 (5, 3): False,
 (7, 7): True,
 (7, 3): False,
 (2, 3): False,
 (8, 5): True,
 (8, 7): True}


The entire matrix also shows which nodes are definitely *not* in the same strongly connected component

### Pathsum by Lehmann

#### Acyclic

In [25]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(0.1))

fsa.add_arc(State(3), Sym('c'), State(5), Real(0.4))
fsa.add_arc(State(3), Sym('a'), State(7), Real(0.7))

fsa.add_arc(State(5), Sym('a'), State(2), Real(0.3))
fsa.add_arc(State(5), Sym('a'), State(7), Real(0.8))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Real'>)

In [26]:
pprint(Pathsum(fsa).lehmann())

{(2, 5): 0.0,
 (5, 2): 0.3,
 (2, 8): 0.1,
 (7, 3): 0.0,
 (3, 3): 1.0,
 (8, 8): 1.0,
 (7, 2): 0.0,
 (3, 8): 0.012,
 (2, 7): 0.0,
 (5, 5): 1.0,
 (7, 7): 1.0,
 (2, 3): 0.0,
 (5, 8): 0.03,
 (3, 7): 1.02,
 (3, 5): 0.4,
 (2, 2): 1.0,
 (8, 5): 0.0,
 (8, 3): 0.0,
 (8, 2): 0.0,
 (3, 2): 0.12,
 (8, 7): 0.0,
 (5, 7): 0.8,
 (7, 5): 0.0,
 (7, 8): 0.0,
 (5, 3): 0.0}


In [27]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

0.0756


#### A self-loop

In [28]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(0.1))

fsa.add_arc(State(3), Sym('c'), State(5), Real(0.4))
fsa.add_arc(State(3), Sym('a'), State(7), Real(0.7))

fsa.add_arc(State(5), Sym('a'), State(2), Real(0.3))
fsa.add_arc(State(5), Sym('a'), State(5), Real(0.9))
fsa.add_arc(State(5), Sym('a'), State(7), Real(0.8))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Real'>)

In [29]:
pprint(Pathsum(fsa).lehmann())

{(2, 8): 0.1,
 (3, 5): 4.000000000000001,
 (8, 2): 0.0,
 (5, 3): 0.0,
 (7, 3): 0.0,
 (7, 2): 0.0,
 (8, 5): 0.0,
 (7, 5): 0.0,
 (5, 5): 10.000000000000002,
 (5, 8): 0.3,
 (5, 7): 8.000000000000002,
 (8, 3): 0.0,
 (3, 8): 0.12,
 (2, 5): 0.0,
 (3, 7): 3.900000000000001,
 (2, 3): 0.0,
 (7, 7): 1.0,
 (8, 8): 1.0,
 (3, 2): 1.2,
 (2, 2): 1.0,
 (8, 7): 0.0,
 (3, 3): 1.0,
 (5, 2): 3.0,
 (2, 7): 0.0,
 (7, 8): 0.0}


In [30]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

0.378


#### A cycle

In [31]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(8), Real(0.1))

fsa.add_arc(State(3), Sym('c'), State(5), Real(0.4))
fsa.add_arc(State(3), Sym('a'), State(7), Real(0.7))

fsa.add_arc(State(5), Sym('a'), State(2), Real(0.3))

fsa.add_arc(State(7), Sym('a'), State(5), Real(0.3))

fsa.add_arc(State(8), Sym('a'), State(7), Real(0.3))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Real'>)

In [32]:
pprint(Pathsum(fsa).lehmann())

{(5, 8): 0.030081219292089,
 (8, 8): 1.002707309736288,
 (7, 3): 0.0,
 (7, 8): 0.009024365787627,
 (7, 5): 0.300812192920886,
 (7, 2): 0.090243657876266,
 (2, 5): 0.009024365787627,
 (3, 2): 0.183495437681741,
 (7, 7): 1.002707309736288,
 (2, 7): 0.030081219292089,
 (2, 3): 0.0,
 (5, 5): 1.002707309736288,
 (5, 7): 0.009024365787627,
 (2, 8): 0.100270730973629,
 (5, 2): 0.300812192920886,
 (3, 5): 0.611651458939136,
 (3, 3): 1.0,
 (8, 3): 0.0,
 (8, 7): 0.300812192920886,
 (5, 3): 0.0,
 (2, 2): 1.002707309736288,
 (3, 8): 0.018349543768174,
 (8, 5): 0.090243657876266,
 (3, 7): 0.705504863130452,
 (8, 2): 0.02707309736288}


In [33]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

0.064349744309636


### Shortest paths by Lehmann (Floyd-Warshall)

#### Acyclic

In [34]:
fsa = FSA(R=Tropical)

fsa.add_arc(State(2), Sym('a'), State(8), Tropical(1))

fsa.add_arc(State(3), Sym('c'), State(5), Tropical(4))
fsa.add_arc(State(3), Sym('a'), State(7), Tropical(7))

fsa.add_arc(State(5), Sym('a'), State(2), Tropical(3))
fsa.add_arc(State(5), Sym('a'), State(7), Tropical(8))

fsa.set_I(State(3), Tropical(3))
fsa.set_F(State(2), Tropical(4))
fsa.set_F(State(7), Tropical(2))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Tropical'>)

In [35]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.VITERBI))

Tropical(12)


In [36]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

Tropical(12)


In [37]:
pprint(Pathsum(fsa).lehmann())

{(5, 5): Tropical(0.0),
 (8, 7): Tropical(inf),
 (2, 8): Tropical(1),
 (2, 7): Tropical(inf),
 (5, 8): Tropical(4),
 (8, 3): Tropical(inf),
 (7, 5): Tropical(inf),
 (2, 3): Tropical(inf),
 (8, 5): Tropical(inf),
 (3, 8): Tropical(8),
 (8, 2): Tropical(inf),
 (2, 5): Tropical(inf),
 (7, 2): Tropical(inf),
 (7, 8): Tropical(inf),
 (7, 3): Tropical(inf),
 (3, 5): Tropical(4),
 (5, 3): Tropical(inf),
 (3, 7): Tropical(7),
 (5, 7): Tropical(8),
 (5, 2): Tropical(3),
 (3, 3): Tropical(0.0),
 (2, 2): Tropical(0.0),
 (8, 8): Tropical(0.0),
 (7, 7): Tropical(0.0),
 (3, 2): Tropical(7)}


#### A self-loop

In [43]:
fsa = FSA(R=Tropical)

fsa.add_arc(State(2), Sym('a'), State(8), Tropical(1))

fsa.add_arc(State(3), Sym('c'), State(5), Tropical(4))
fsa.add_arc(State(3), Sym('a'), State(7), Tropical(7))

fsa.add_arc(State(5), Sym('a'), State(2), Tropical(3))
fsa.add_arc(State(5), Sym('a'), State(5), Tropical(9))
fsa.add_arc(State(5), Sym('a'), State(7), Tropical(8))

fsa.set_I(State(3), Tropical(3))
fsa.set_F(State(2), Tropical(4))
fsa.set_F(State(7), Tropical(2))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Tropical'>)

In [44]:
pprint(Pathsum(fsa).lehmann())

{(5, 3): Tropical(inf),
 (7, 8): Tropical(inf),
 (5, 8): Tropical(4),
 (8, 3): Tropical(inf),
 (8, 2): Tropical(inf),
 (7, 3): Tropical(inf),
 (3, 5): Tropical(4),
 (2, 8): Tropical(1),
 (2, 5): Tropical(inf),
 (3, 2): Tropical(7),
 (8, 7): Tropical(inf),
 (5, 2): Tropical(3),
 (3, 3): Tropical(0.0),
 (8, 5): Tropical(inf),
 (7, 5): Tropical(inf),
 (7, 7): Tropical(0.0),
 (7, 2): Tropical(inf),
 (8, 8): Tropical(0.0),
 (3, 8): Tropical(8),
 (2, 7): Tropical(inf),
 (5, 7): Tropical(8),
 (2, 2): Tropical(0.0),
 (5, 5): Tropical(0.0),
 (3, 7): Tropical(7),
 (2, 3): Tropical(inf)}


In [45]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

Tropical(12)


We see that since the Tropical semiring is 0-closed, the self loop does not affect the pathsum

#### A Cycle

In [46]:
fsa = FSA(R=Tropical)

fsa.add_arc(State(2), Sym('a'), State(8), Tropical(1))

fsa.add_arc(State(3), Sym('c'), State(5), Tropical(4))
fsa.add_arc(State(3), Sym('a'), State(7), Tropical(7))

fsa.add_arc(State(5), Sym('a'), State(2), Tropical(3))

fsa.add_arc(State(7), Sym('a'), State(5), Tropical(3))

fsa.add_arc(State(8), Sym('a'), State(7), Tropical(3))

fsa.set_I(State(3), Tropical(3))
fsa.set_F(State(2), Tropical(4))
fsa.set_F(State(7), Tropical(2))

fsa

WFSA(5 states, <class 'rayuela.base.semiring.Tropical'>)

In [47]:
pprint(Pathsum(fsa).lehmann())

{(7, 7): Tropical(0.0),
 (8, 2): Tropical(9),
 (3, 8): Tropical(8),
 (5, 3): Tropical(inf),
 (2, 3): Tropical(inf),
 (7, 3): Tropical(inf),
 (5, 7): Tropical(7),
 (3, 5): Tropical(4),
 (8, 3): Tropical(inf),
 (5, 8): Tropical(4),
 (3, 2): Tropical(7),
 (2, 2): Tropical(0.0),
 (2, 5): Tropical(7),
 (8, 5): Tropical(6),
 (3, 3): Tropical(0.0),
 (2, 7): Tropical(4),
 (8, 8): Tropical(0.0),
 (2, 8): Tropical(1),
 (3, 7): Tropical(7),
 (7, 8): Tropical(7),
 (8, 7): Tropical(3),
 (7, 2): Tropical(6),
 (7, 5): Tropical(3),
 (5, 5): Tropical(0.0),
 (5, 2): Tropical(3)}


In [48]:
pprint(Pathsum(fsa).pathsum(strategy=Strategy.LEHMANN))

Tropical(12)


Again, the cycle does not affect the pathsum

## Example 4

Connected components by hand

In [28]:
fsa = FSA(R=Real)

fsa.add_arc(State(2), Sym('a'), State(5), Real(0.1))

fsa.add_arc(State(3), Sym('a'), State(6), Real(0.1))
fsa.add_arc(State(3), Sym('b'), State(1), Real(0.7))

fsa.add_arc(State(1), Sym('c'), State(3), Real(0.3))
fsa.add_arc(State(1), Sym('c'), State(5), Real(0.3))
fsa.add_arc(State(1), Sym('b'), State(4), Real(0.1))

fsa.add_arc(State(4), Sym('a'), State(2), Real(0.5))

fsa.add_arc(State(5), Sym('a'), State(7), Real(0.8))

fsa.add_arc(State(6), Sym('a'), State(7), Real(0.7))
fsa.add_arc(State(6), Sym('a'), State(3), Real(0.1))

fsa.add_arc(State(7), Sym('a'), State(8), Real(0.2))

fsa.add_arc(State(8), Sym('a'), State(7), Real(0.2))
fsa.add_arc(State(8), Sym('a'), State(2), Real(0.2))

fsa.set_I(State(3), Real(0.3))
fsa.set_F(State(2), Real(0.4))
fsa.set_F(State(7), Real(0.2))

fsa

WFSA(8 states, <class 'rayuela.base.semiring.Real'>)

In [31]:
list(fsa.finish())

[3, 1, 4, 6, 7, 8, 2, 5]